In [ ]:
def cal_TDI(esn, X_test, n_nodes, n_nodes_dim, target_i, target_j, n_rep, n_washout, n_evo, j_threshold = 0.1):
    Error = np.zeros((n_rep, n_nodes, n_nodes))
    for rep in range(n_rep):
        offset = np.random.randint(X_test.shape[0] - n_evo - n_washout)
        X_test_washout = X_test[- n_washout + offset : offset]
        Y = X_test[offset : offset + n_evo]
        
        r0 = esn.open_loop(X_test_washout, np.zeros(esn.n_units))[-1]
        Yh = esn.evolve(r0, N_evo)
        
        E = np.zeros((n_nodes, n_nodes))
        for i in range(n_nodes):
            for j in range(n_nodes):
                if i == j:
                    continue
                targetI = n_nodes_dim*i + target_i
                targetJ = n_nodes_dim*j + target_j

                jmean = np.mean(Yh[:,targetJ], axis=0)
                if np.abs(jmean) < j_threshold:
                    E[i,j] = -1
                    continue

                Yhji = esn.evolve_edge_removal(r0, targetJ, targetI, N_evo)

                d = (Yh - Yhji)[:,targetI]
                E[i,j] = np.sum(d ** 2) / N_evo / jmean

        Error[rep] = E

    TDI = np.zeros((n_nodes, n_nodes))
    for i in range(n_nodes):
        for j in range(n_nodes):
            if i == j:
                continue
            t = Error[:,i,j]
            e = t[t > 0]
            if e.shape[0] == 0:
                print(f'warning: too large threshold, i:{i},j:{j} no error data')
                TDI[i,j] = 0
            else:
                TDI[i,j] = np.mean(e)
                
    return TDI